In [75]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import numpy as np
import torch
from string import punctuation
import os

In [81]:
with open('C:/Users/kaush/Desktop/aclImdb/train/trainReviews.txt', 'r', encoding="utf8") as f:
    reviews = f.read()
labels = pd.read_csv('C:/Users/kaush/Desktop/aclImdb/train/IMDBValuesTrain.txt', sep='delimiter', names=['Score'], engine = 'python')
    

In [69]:
reviews = reviews.lower()

In [85]:
all_text = ''.join([c for c in reviews if c not in punctuation])

reviews_split = all_text.split('\n')

labels = labels.to_numpy()


[[ 3]
 [ 4]
 [ 4]
 ...
 [10]
 [ 8]
 [ 7]]


In [71]:
all_text2 = ' '.join(reviews_split)

words = all_text2.split()

count_words = Counter(words)

total_words = len(words)
sorted_words = count_words.most_common(total_words)

In [72]:
vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}



In [73]:
reviews_int = []
for reviews in reviews_split:
    r = [vocab_to_int[w] for w in reviews.split()]
    reviews_int.append(r)


[[63, 4, 3, 132, 36, 43, 7619, 1364, 15, 3, 4623, 499, 45, 16, 3, 614, 134, 11, 6, 3, 1274, 453, 4, 1749, 219, 3, 10833, 7739, 309, 6, 676, 77, 33, 2082, 1093, 3008, 32, 1, 918, 57550, 4, 29, 5405, 458, 8, 2587, 1749, 1, 218, 61, 16, 56, 802, 1292, 823, 244, 8, 40, 99, 126, 1462, 53, 143, 35, 1, 1075, 138, 27, 676, 126, 1, 14098, 406, 58, 93, 2231, 288, 770, 5, 3, 837, 14099, 20, 3, 1667, 679, 29, 124, 70, 22, 226, 102, 16, 46, 49, 607, 32, 707, 84, 32064, 32065, 707, 389, 3467, 13147, 2, 17217, 8386, 68, 27, 107, 3309], [4487, 13603, 499, 14, 3, 3468, 154, 8540, 13148, 1656, 6, 4844, 57, 16, 4425, 5823, 135, 11996, 5, 1003, 4941, 2991, 4488, 564, 1347, 36, 6, 1514, 97, 3, 737, 4, 57551, 5, 24, 3531, 7, 10834, 4, 8, 106, 3040, 5, 1, 1048, 14, 3, 4489, 78, 20, 2069, 6, 4488, 578, 2769, 7307, 43770, 39, 500, 1, 8540, 57552, 291, 126, 14, 4287, 18, 17218, 1, 1656, 6, 14623, 32, 1, 43771, 24234, 644, 26241, 24, 104, 24235, 15187, 16479, 20049, 2588, 486, 22572, 36, 3725, 1, 6090, 1030, 45,

In [86]:
# Change Review Score to Binary Score
encoded_labels = [0 if label <= 5 else 1 for label in labels]
encoded_labels = np.array(encoded_labels)
print(encoded_labels)

[0 0 0 ... 1 1 1]


In [88]:
reviews_len = [len(x) for x in reviews_int]
reviews_int = [reviews_int[i] for i, l in enumerate(reviews_len) if l > 0]
encoded_labels = [encoded_labels[i] for i, l in enumerate(reviews_len) if l > 0]

In [93]:
seq_length = 200
def pad_features(reviews_int, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's or truncated to the input seq_length.
    '''
    features = np.zeros((len(reviews_int), seq_length), dtype = int)
    
    for i, review in enumerate(reviews_int):
        review_len = len(review)
        
        if review_len <= seq_length:
            zeroes = list(np.zeros(seq_length-review_len))
            new = zeroes+review
        elif review_len > seq_length:
            new = review[0:seq_length]
        
        features[i,:] = np.array(new)
    
    return features

pad_features(reviews_int, seq_length)

array([[    0,     0,     0, ...,    27,   107,  3309],
       [ 4487, 13603,   499, ...,     1, 32068,  1725],
       [    0,     0,     0, ...,  1853,    15,   625],
       ...,
       [    9,   236,  2635, ...,    26,   598,    20],
       [    0,     0,     0, ...,  3752,    14,    73],
       [    0,     0,     0, ...,    80,    96,    35]])

NameError: name 'features' is not defined

In [88]:
count_words = Counter(df_reviews_train['Review'])
total_words = len(df_reviews_train['Review'])
sorted_words = count_words.most_common(total_words)
print(count_words)

TypeError: unhashable type: 'list'

[[   11    49  7487 ...     0     0     0]
 [ 2899  1542  2069 ...  4362 13466  5454]
 [    3  3459    53 ...     0     0     0]
 ...
 [   71    14    19 ...   976   107  9625]
 [  845   192    66 ...     0     0     0]
 [13545   605   367 ...     0     0     0]]
[[  328 12069  3133 ...     0     0     0]
 [  250   115    10 ...     3    82  2743]
 [   20   667  4855 ...     0     0     0]
 ...
 [   57   228     2 ...  3584    49   202]
 [    2  4551  4047 ...     0     0     0]
 [   99    33    11 ...     0     0     0]]
